## Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Dataset Installation

In [2]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2024-08-05 20:04:07--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.004s  

2024-08-05 20:04:07 (6.78 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2024-08-05 20:04:08--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP re

## Dataset Investigation

In [3]:
input_files=['edgar_allan_poe.txt','robert_frost.txt']

In [4]:
!cat edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!
Time-eaten towers that tremble not!
Resemble nothing that is ours.
Around, by lifting winds forgot,
Resignedly beneath the sky
The melancholy waters lie.
 
No holy rays from heaven come down
On the long night-time of that town,
But light from out the lurid sea
Streams up the turrets silently
Up thrones up long-forgotten bowers
Of scultur'd ivy and stone flowers
Up domes up spires up kingly halls
Up fanes up Babylon-like walls
Up many a melancholy shrine
Whose entablatures intertwine
The mask the viol and the vine.
 
There open temples open graves
Are on a level with the waves
But not the riches there that lie
In each idol's diamond eye,
Not the gaily-jewell'd dead
Te

In [5]:
!cat robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day! 
Yet knowing how way leads on to way
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I,
I took the one less traveled by,
And that has made all the difference.

Whose woods these are I think I know.
His house is in the village, though; 
He will not see me stopping here
To watch his woods fill up with snow.

My little horse must think it queer
To stop without a farmhouse near
Between the woods and frozen lake
The darkest evenin

The first thing we need to do is create samples and corresponding labels and we should remove the whitespaces from the sentences and remove empty lines in poetries.

In [32]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [33]:
print(input_texts)
print(labels)

['lo death hath reard himself a throne', 'in a strange city all alone', 'far down within the dim west', 'where the good and the bad and the worst and the best', 'have gone to their eternal rest', 'there shrines and palaces and towers', 'are not like any thing of ours', 'oh no o no ours never loom', 'to heaven with that ungodly gloom', 'timeeaten towers that tremble not', 'resemble nothing that is ours', 'around by lifting winds forgot', 'resignedly beneath the sky', 'the melancholy waters lie', 'no holy rays from heaven come down', 'on the long nighttime of that town', 'but light from out the lurid sea', 'streams up the turrets silently', 'up thrones up longforgotten bowers', 'of sculturd ivy and stone flowers', 'up domes up spires up kingly halls', 'up fanes up babylonlike walls', 'up many a melancholy shrine', 'whose entablatures intertwine', 'the mask the viol and the vine', 'there open temples open graves', 'are on a level with the waves', 'but not the riches there that lie', 'in e

## Train-Test Split

In this step, we need to split our data set into a training part and a test part. We are going to use a train ratio of 0.8 and a test ratio of 0.2 for this operation.

In [34]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [35]:
print(f"Number of training samples: {len(train_text)}")
print(f"Number of test samples: {len(test_text)}")

Number of training samples: 1615
Number of test samples: 539


Let we inspect 5 sample sentence and corresponding labels together.

In [36]:
train_text[:5]

['i like your going to be you said just now',
 'the night the bones came up the cellarstairs',
 'wearing its own deep feeling as a crown',
 'not lupine living on sand and drouth',
 'this all this was in the olden']

In [37]:
Ytrain[:5]

[1, 1, 0, 1, 0]

## Word to index mapping

In [38]:
idx = 1
word2idx = {'<unk>': 0}

In [39]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [40]:
word2idx

{'<unk>': 0,
 'i': 1,
 'like': 2,
 'your': 3,
 'going': 4,
 'to': 5,
 'be': 6,
 'you': 7,
 'said': 8,
 'just': 9,
 'now': 10,
 'the': 11,
 'night': 12,
 'bones': 13,
 'came': 14,
 'up': 15,
 'cellarstairs': 16,
 'wearing': 17,
 'its': 18,
 'own': 19,
 'deep': 20,
 'feeling': 21,
 'as': 22,
 'a': 23,
 'crown': 24,
 'not': 25,
 'lupine': 26,
 'living': 27,
 'on': 28,
 'sand': 29,
 'and': 30,
 'drouth': 31,
 'this': 32,
 'all': 33,
 'was': 34,
 'in': 35,
 'olden': 36,
 'arent': 37,
 'afraid': 38,
 'of': 39,
 'him': 40,
 'whats': 41,
 'that': 42,
 'gun': 43,
 'for': 44,
 'way': 45,
 'he': 46,
 'did': 47,
 'life': 48,
 'once': 49,
 'but': 50,
 'time': 51,
 'town': 52,
 'is': 53,
 'no': 54,
 'more': 55,
 'with': 56,
 'pearl': 57,
 'ruby': 58,
 'glowing': 59,
 'when': 60,
 'there': 61,
 'water': 62,
 'cellar': 63,
 'spring': 64,
 'dont': 65,
 'believe': 66,
 'me': 67,
 'cased': 68,
 'world': 69,
 'gone': 70,
 'high': 71,
 'tone': 72,
 'spirit': 73,
 'which': 74,
 'hath': 75,
 'striven': 76,
 

In [41]:
len(word2idx)

2505

## Convert string words to integer

In [42]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [43]:
train_text_int[100:105]


[[35, 381, 39, 210, 52, 382, 383, 245, 40, 384],
 [42, 56, 23, 385, 386, 387, 388, 111, 389],
 [390, 391, 30, 285],
 [5, 392, 179, 325, 393, 15, 56, 267],
 [100, 355, 331, 332, 110, 61, 6, 113]]

In [44]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [45]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In the transition matrix, we need to normalize each row. In order to do this, we have to divide each value by the sum of all values.

In [46]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

Here is the normalized version of transtiopn matrix and initial matrix

In [47]:
A0

array([[0.0003992 , 0.0003992 , 0.0003992 , ..., 0.0003992 , 0.0003992 ,
        0.0003992 ],
       [0.000392  , 0.000392  , 0.000392  , ..., 0.000392  , 0.000392  ,
        0.000392  ],
       [0.00039793, 0.00039793, 0.00039793, ..., 0.00039793, 0.00039793,
        0.00039793],
       ...,
       [0.0003992 , 0.0003992 , 0.0003992 , ..., 0.0003992 , 0.0003992 ,
        0.0003992 ],
       [0.0003992 , 0.0003992 , 0.0003992 , ..., 0.0003992 , 0.0003992 ,
        0.0003992 ],
       [0.0003992 , 0.0003992 , 0.0003992 , ..., 0.0003992 , 0.0003992 ,
        0.0003992 ]])

In [48]:
pi0

array([0.00032841, 0.00361248, 0.00197044, ..., 0.00032841, 0.00032841,
       0.00032841])

We are applying one-smoothing to normalized matrixes.



In [49]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [50]:
print(logA0)
print(logpi0)

[[-7.82604401 -7.82604401 -7.82604401 ... -7.82604401 -7.82604401
  -7.82604401]
 [-7.84424072 -7.84424072 -7.84424072 ... -7.84424072 -7.84424072
  -7.84424072]
 [-7.82923254 -7.82923254 -7.82923254 ... -7.82923254 -7.82923254
  -7.82923254]
 ...
 [-7.82604401 -7.82604401 -7.82604401 ... -7.82604401 -7.82604401
  -7.82604401]
 [-7.82604401 -7.82604401 -7.82604401 ... -7.82604401 -7.82604401
  -7.82604401]
 [-7.82604401 -7.82604401 -7.82604401 ... -7.82604401 -7.82604401
  -7.82604401]]
[-8.02125618 -5.62336091 -6.22949671 ... -8.02125618 -8.02125618
 -8.02125618]


In [51]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33436532507739936, 0.6656346749226006)

In [52]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [53]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [54]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9938080495356038


In [55]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8311688311688312


In [56]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [57]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 530,   10],
       [   0, 1075]])

In [58]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 96,  82],
       [  9, 352]])

In [59]:
f1_score(Ytrain, Ptrain)

0.9953703703703703

In [60]:
f1_score(Ytest, Ptest)


0.8855345911949686